### Prerequisites

To run this .NET Interactive run book, you need a create an AppRegistration in Azure AD with the following settings
- Authentication -> Platform: Mobile and Desktop
- Authentication -> Platform -> RedirectUris: Msal...
- Authentication -> Supported Account Types: Single Tenant
- Authentication -> Advanced Settings -> Allow public client flows: True
- Press **Save**

- API Permissions -> Add a Permission -> Delegated Permisions -> MS Graph: `Application.ReadWrite.All`

We start by importing the necessary NuGet packages to be used throughout this runbook

In [ ]:
#r "nuget:Microsoft.Graph,3.32.0"
#r "nuget:Microsoft.Identity.Client,4.31.0"
#r "nuget:Microsoft.Graph.Auth,1.0.0-preview.7"

### Create the MSAL Client
Ensure that you add the appropriate `usings` so that we can interact with Graph and AAD.

Instantiate the MSAL Client to authenticate against AAD and get the right Graph Permissions.
The MSAL Client requires the 2 following properties:
- `ClientID` (From the Overview tab of your AAD App Registration)
- `TenantID` (From the Overview tab. Can be the tenant id [Guid] or the tenant name)

In [ ]:
using Microsoft.Identity.Client;
using System.Threading.Tasks;
using Microsoft.Graph.Auth;

public static DeviceCodeProvider GetMsalClient()
{
    var clientId = "18e9ac17-ae4b-4cea-aeb8-cb25a68e6867";
    var tenantId = "b55f0c51-61a7-45c3-84df-33569b247796";
    
    IPublicClientApplication publicClientApplication = PublicClientApplicationBuilder
        .Create(clientId)
        .WithTenantId(tenantId)
        .Build();
    var scopes = new string[]{"Files.ReadWrite"};
    return new DeviceCodeProvider(publicClientApplication, scopes);
}

### Upload a large file using the Graph SDK

Make sure to update the `filepath` with your file location.

In [ ]:
using Microsoft.Graph;

var filePath = @"C:\Users\chmatsk\Downloads\2021-06-01 16-29-34.mp4";
var graphClient = new GraphServiceClient(GetMsalClient());

using (var fileStream = System.IO.File.OpenRead(filePath))
{
    // Use properties to specify the conflict behavior
    // in this case, replace
    var uploadProps = new DriveItemUploadableProperties
    {
        ODataType = null,
        AdditionalData = new Dictionary<string, object>
        {
            { "@microsoft.graph.conflictBehavior", "replace" }
        }
    };

    var uploadSession = await graphClient.Me.Drive.Root
        .ItemWithPath(System.IO.Path.GetFileName(filePath))
        .CreateUploadSession(uploadProps)
        .Request()
        .PostAsync();

    // Max slice size must be a multiple of 320 KiB
    int maxSliceSize = 320 * 1024;
    var fileUploadTask =
        new LargeFileUploadTask<DriveItem>(uploadSession, fileStream, maxSliceSize);

    // Create a callback that is invoked after each slice is uploaded
    IProgress<long> progress = new Progress<long>(prog => {
        Console.WriteLine($"Uploaded {prog} bytes of {fileStream.Length} bytes");
    });

    try
    {
        // Upload the file
        var uploadResult = await fileUploadTask.UploadAsync(progress);

        if (uploadResult.UploadSucceeded)
        {
            // The ItemResponse object in the result represents the
            // created item.
            Console.WriteLine($"Upload complete, item ID: {uploadResult.ItemResponse.Id}");
        }
        else
        {
            Console.WriteLine("Upload failed");
        }
    }
    catch (ServiceException ex)
    {
        Console.WriteLine($"Error uploading: {ex.ToString()}");
    }
}